In [1]:
from transformers_interpret import SequenceClassificationExplainer

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
import torch.nn as nn
import pytorch_lightning as pl 
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

from transformers import (
    BertForMaskedLM,
    AutoTokenizer,
    AutoConfig,
    BertModel, 
    BertConfig, 
    BertTokenizer, 
    BertForSequenceClassification
)

LABEL_COLUMNS = ['card_delivery_estimate','cash_withdrawal_charge','balance_not_updated_after_cheque_or_cash_deposit','receiving_money','edit_personal_details','card_arrival','card_acceptance','fiat_currency_support','declined_cash_withdrawal','country_support','topping_up_by_card','wrong_exchange_rate_for_cash_withdrawal','Refund_not_showing_up','getting_virtual_card','visa_or_mastercard','balance_not_updated_after_bank_transfer','automatic_top_up','age_limit','transfer_fee_charged','card_not_working','lost_or_stolen_phone','verify_source_of_funds','card_about_to_expire','verify_top_up','top_up_reverted','pending_top_up','top_up_failed','card_payment_fee_charged','atm_support','pending_cash_withdrawal','pending_card_payment','cancel_transfer','top_up_by_cash_or_cheque','getting_spare_card','request_refund','direct_debit_payment_not_recognised','disposable_card_limits','order_physical_card','failed_transfer','reverted_card_payment?','declined_transfer','extra_charge_on_statement','apple_pay_or_google_pay','get_disposable_virtual_card','card_payment_wrong_exchange_rate','lost_or_stolen_card','activate_my_card','transfer_timing','exchange_via_app','why_verify_identity','top_up_by_card_charge','passcode_forgotten','card_linking','card_swallowed','unable_to_verify_identity','wrong_amount_of_cash_received','beneficiary_not_allowed','top_up_by_bank_transfer_charge','change_pin','verify_my_identity','transaction_charged_twice','top_up_limits','cash_withdrawal_not_recognised','declined_card_payment','exchange_rate','pending_transfer','contactless_not_working','pin_blocked','card_payment_not_recognised','virtual_card_not_working','transfer_not_received_by_recipient','compromised_card','terminate_account','transfer_into_account','exchange_charge','supported_cards_and_currencies','get_physical_card']

def print_model_info(model):
    # Get all of the model's parameters as a list of tuples.
    params = list(model.named_parameters())

    print('The BERT model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

def tokenize_sentences(sentences):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # encode_plus will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the [CLS] token to the start.
        #   (3) Append the [SEP] token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to max_length
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 320,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids,attention_masks        

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

chk_path = PATH
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(chk_path, map_location=torch.device('cpu'))

# config = BertConfig.from_pretrained("bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained(
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = len(LABEL_COLUMNS),
#     state_dict=checkpoint
# )

model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 77
)

print_model_info(model)

#load specified model state

# print(checkpoint["state_dict"].get('bert.embeddings.word_embeddings.num_embeddings'))
model.load_state_dict(checkpoint["state_dict"])
model.eval()
model.to(device)

Loading BERT tokenizer...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [3]:
while(True):
    string = input("How may I help you? Question: ")
    #tokenize inputted sentence to be compatible with BERT inputs
    token_ids,attention_masks = tokenize_sentences([string])
    #get a tensor containing probabilities of inputted sentence being irrelevant or relevant
    model_outputs = (model(token_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device)))
    softmax_layer = torch.nn.Softmax()
    result = softmax_layer(model_outputs[0])
    #identify which output node has higher probability and what that probability is
    prediction = torch.argmax(result).item()
    confidence = torch.max(result).item()
    print("I see. The problem is related to: " + LABEL_COLUMNS[prediction].replace("_", ' ') + " with {:.2f}% confident".format(confidence*100))
    print("-"*80)

How may I help you? Question: I lost my password
I see. The problem is related to: passcode forgotten with 99.14% confident
--------------------------------------------------------------------------------
How may I help you? Question: I bought an item and noticed the exchange rate was not correct.
I see. The problem is related to: card payment wrong exchange rate with 98.99% confident
--------------------------------------------------------------------------------
How may I help you? Question: Delete this acccount!
I see. The problem is related to: terminate account with 96.78% confident
--------------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [ ]:
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

while(True):
    string = input("How may I help you? Question: ")
    #tokenize inputted sentence to be compatible with BERT inputs
    token_ids,attention_masks = tokenize_sentences([string])
    #get a tensor containing probabilities of inputted sentence being irrelevant or relevant
    model_outputs = (model(token_ids.to(device), token_type_ids=None, attention_mask=attention_masks.to(device)))
    softmax_layer = torch.nn.Softmax()
    result = softmax_layer(model_outputs[0])
    #identify which output node has higher probability and what that probability is
    prediction = torch.argmax(result).item()
    confidence = torch.max(result).item()
    print("I see. The problem is related to: " + LABEL_COLUMNS[prediction].replace("_", ' ') + " with {:.2f}% confident".format(confidence*100))
    word_attributions = multiclass_explainer(text=string)
    print("-"*80)
    print(word_attributions)
    html = multiclass_explainer.visualize()
    print(html)
    print("*"*100)
    print("*"*100)
    

How may I help you? Question: I lost my password.
I see. The problem is related to: passcode forgotten with 99.13% confident
--------------------------------------------------------------------------------
[('[CLS]', 0.0), ('I', 0.0562481692013938), ('lost', 0.05805628611688607), ('my', -0.1091432691471927), ('password', 0.9594285902482548), ('.', -0.24708326150898116), ('[SEP]', 0.0)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
51,LABEL_51 (0.99),LABEL_51,0.72,[CLS] I lost my password . [SEP]


<IPython.core.display.HTML object>
****************************************************************************************************
****************************************************************************************************
How may I help you? Question: Can you assist me with the auto top up?
I see. The problem is related to: automatic top up with 99.36% confident
--------------------------------------------------------------------------------
[('[CLS]', 0.0), ('Can', 0.009763619382970408), ('you', 0.10598783501102706), ('assist', 0.060547744330326655), ('me', 0.048150871218004564), ('with', 0.06964565376013586), ('the', 0.030727987086627426), ('auto', 0.9850763005755512), ('top', -0.009123880987533884), ('up', -0.036557802796218355), ('?', 0.07139294391204447), ('[SEP]', 0.0)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
16,LABEL_16 (0.99),LABEL_16,1.34,[CLS] Can you assist me with the auto top up ? [SEP]


<IPython.core.display.HTML object>
****************************************************************************************************
****************************************************************************************************
